In [2]:
# this notebook is used to create embs of all contracts for this okved
import pandas as pd
from pathlib import Path
import sqlite3
import joblib
from transformers import AutoModel, AutoTokenizer
from data.build_okved_dataset import embed_bert_cls
from tqdm import tqdm
import numpy as np

c:\Users\egrli\envs\okved_embeddings\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Добавим следующую информацию:\
В узлы: 
1. `эмбеддинги госзакупок`

В ребра: 
1. `Объем госзакупок между ОКВЭДами`

In [3]:
OKVED_ROOT = Path(r'C:\Users\egrli\FU_Projects\Курсач\intelligent_okved_embeddings\data\okved2')
DB_ROOT = Path(r"C:\Users\egrli\FU_Projects\Курсач\intelligent_okved_embeddings\data\db")
okved_data = pd.read_csv(OKVED_ROOT / 'okved_2014_w_sections.csv',
                            index_col=0,
                            dtype={'code': str,
                                'code_okved': str,
                                'parent_code': str,
                                'native_code': str})

In [4]:
con = sqlite3.connect(DB_ROOT / "okved_20230329.sqlite3")
cur = con.cursor()

In [5]:
tables = cur.execute("""
  SELECT * FROM sqlite_master where type='table'
  -- LIMIT 5
""").fetchall()
table_names = list(
    map(
        lambda x: x[1],
        tables
    )
)
table_to_info = {}
for table_name in table_names:
    table_to_info[table_name] = pd.DataFrame(cur.execute(f"""
PRAGMA table_info({table_name});
""").fetchall()).drop(columns=[0, 3, 4, 5])

In [6]:
table_to_info.keys()

dict_keys(['natural_person', 'supplier', 'okved', 'legal_entity', 'product', 'company_to_okved', 'company_to_natural_member', 'company_to_legal_member', 'procurement'])

## Эмбеддинги узлов окведов

In [7]:
query = """
  SELECT 
  le.main_okved_code,
  GROUP_CONCAT(DISTINCT p.object) --p.object=p.name
  --p.method, 
  --p.name, 
  --p.executor_inn, 
  FROM procurement AS p
  INNER JOIN legal_entity AS le 
  ON p.executor_inn=le.inn
  WHERE le.main_okved_code IS NOT NULL
  GROUP BY le.main_okved_code
  --LIMIT 10; 
"""
okved_to_agg_info_orders = pd.DataFrame(cur.execute(query).fetchall(), columns=['okved', 'agg_contracts'])

In [8]:
okved_to_agg_info_orders = okved_to_agg_info_orders.merge(okved_data, how='inner', left_on='okved', right_on='native_code')[["okved", "agg_contracts"]]

In [9]:
okved_to_number_mapping = {code : i for i, code in enumerate(okved_to_agg_info_orders['okved'].values.tolist())}

In [10]:
joblib.dump(value=okved_to_number_mapping, filename=OKVED_ROOT / 'okved_to_number_mapping.pickle');

In [11]:
model_name = "cointegrated/rubert-tiny2" # эмбеддинги- 312 (неужели мы в узлы больше ничего не берем)
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:
embds = []
for text in tqdm(okved_to_agg_info_orders['agg_contracts']):
    embds.append(embed_bert_cls(text=text, model=model, tokenizer=tokenizer, truncation=True))
embds = np.concatenate(embds)
np.save(file=OKVED_ROOT / "okved_contact_emb.npy", arr=embds)

100%|██████████| 549/549 [00:47<00:00, 11.58it/s]


## Объем средств в гос-контрактах между оквэдами

100%|██████████| 22915/22915 [00:00<00:00, 24476.05it/s]


In [48]:
gos_contracts_embeddings.shape

(2637, 312)

In [49]:
okved_embeddings.shape

torch.Size([2637, 312])

In [17]:
g = build_graph(okved_embeddings, cc_edges, clf_edges, extra_edges)
with open(GRAPH_ROOT / 'okved_graph.pickle', 'wb') as fp:
    pickle.dump(g, fp)

In [75]:
query = """
  SELECT 
  SUM(start_price), COUNT(start_price), AVG(start_price),
  y.main_okved_code, z.main_okved_code
  FROM procurement AS x
  INNER JOIN legal_entity AS y 
  ON x.executor_inn=y.inn
  INNER JOIN legal_entity as z
  ON x.customer_inn=z.inn 
  WHERE y.main_okved_code IS NOT NULL 
  AND z.main_okved_code IS NOT NULL
  GROUP BY y.main_okved_code, z.main_okved_code
"""
start_prices = pd.DataFrame(cur.execute(query).fetchall(), columns=[
    "sum_gos", "count_gos", "avg_gos", "executor_okved", "customer_okved"
])

In [78]:
start_prices.to_csv(r"C:\Users\egrli\FU_Projects\Курсач\intelligent_okved_embeddings\data\okved2\start_prices.csv")

100%|██████████| 22915/22915 [00:00<00:00, 23752.88it/s]


In [116]:
g = build_graph(okved_embeddings, cc_edges, clf_edges, extra_edges, gos_contracts_edges)


In [123]:
with open(GRAPH_ROOT / 'okved_graph.pickle', 'wb') as fp:
    pickle.dump(g, fp)